In [ ]:
import cv2
import numpy as np
 
# https://docs.opencv.org/4.x/d2/d55/group__bgsegm.html 
backSub = cv2.createBackgroundSubtractorMOG2()
# backSub = cv2.createBackgroundSubtractorKNN()
# backSub = cv2.bgsegm.createBackgroundSubtractorCNT()
# backSub = cv2.bgsegm.createBackgroundSubtractorGSOC()
 
capture = cv2.VideoCapture(cv2.samples.findFileOrKeep('./video/vtest.avi'))
 
lastframe = None
while True:
    ret, frame = capture.read()
    if frame is None:
        break

    if lastframe is None:
        lastframe = frame
        
    cv2.imshow('Frame', frame)

    # 简单的前后两帧相减
    fgMask = np.abs(frame.astype(float) - lastframe.astype(float)).astype(np.uint8)
    cv2.imshow('SimpleSub', fgMask)
    lastframe = frame

    # 直接调用 apply 方法
    fgMask2 = backSub.apply(frame)

    # 添加矩形框，比较粗糙，可以进一步细节调整（如形态学等操作）
    contours, hierachy = cv2.findContours(fgMask2, cv2.RETR_LIST, method=cv2.CHAIN_APPROX_NONE)
    for i in range(len(contours)):
        now_contours = contours[i] # (n, 1, 2)
        if (cv2.contourArea(now_contours) > 100):
            x0,y0,w0,h0 = cv2.boundingRect(now_contours)
            cv2.rectangle(fgMask2, (x0,y0), (x0+w0,y0+h0), 255)
    cv2.imshow('BackGroundSub', fgMask2)

    cv2.waitKey(1)